In [ ]:
# This approch will be to load in our CVE dataset, and the DiverseVul Dataset which willhave 'func' - function, 'target' - 1 or 0 value for vul or not, 'cwe' - A category code, 'message' - commit message.
# We then take this data and summarize each 'func' with an LLM that describes and vulenabilites in the code, and then trys to match the vulnerability summary to the CVE summary.
# Then after we mark each 'func' as vulnerable or not we compare the 'target' label with the prediction and record the recall, accuracy, and F1 scores.

In [ ]:
### 1. Install the Necessary Libraries

import json
import pandas as pd
from transformers import pipeline
from sklearn.metrics import accuracy_score, recall_score, f1_score, classification_report
from transformers import AutoTokenizer, AutoModel
!pip install transformers torch
!pip install --upgrade tensorflow-probability
!pip install --upgrade --quiet openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.8/326.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86

In [ ]:
import pandas as pd
import json

# Define the path to your JSON file
code_data = "/content/drive/MyDrive/processedData.json"
cve_summaries_path = "/content/drive/MyDrive/training_data.csv"

# This will store the corrected JSON objects
corrected_data = [] # code_data

# Read the file and convert each JSON object correctly
with open(code_data, 'r') as file:
    for line in file:
        try:
            # Try to parse each line as a separate JSON object
            json_object = json.loads(line)
            corrected_data.append(json_object)
        except json.JSONDecodeError:
            print("Error decoding JSON from line:", line)

print(corrected_data[0])

# Load CSV data
df = pd.read_csv(cve_summaries_path)

print("\nCSV Data Sample:")
print(df.head())  # Display the first 5 rows of the DataFrame

{'func': 'int _gnutls_ciphertext2compressed(gnutls_session_t session,\n\t\t\t\t  opaque * compress_data,\n\t\t\t\t  int compress_size,\n\t\t\t\t  gnutls_datum_t ciphertext, uint8 type)\n{\n    uint8 MAC[MAX_HASH_SIZE];\n    uint16 c_length;\n    uint8 pad;\n    int length;\n    mac_hd_t td;\n    uint16 blocksize;\n    int ret, i, pad_failed = 0;\n    uint8 major, minor;\n    gnutls_protocol_t ver;\n    int hash_size =\n\t_gnutls_hash_get_algo_len(session->security_parameters.\n\t\t\t\t  read_mac_algorithm);\n\n    ver = gnutls_protocol_get_version(session);\n    minor = _gnutls_version_get_minor(ver);\n    major = _gnutls_version_get_major(ver);\n\n    blocksize = _gnutls_cipher_get_block_size(session->security_parameters.\n\t\t\t\t\t      read_bulk_cipher_algorithm);\n\n    /* initialize MAC \n     */\n    td = mac_init(session->security_parameters.read_mac_algorithm,\n\t\t  session->connection_state.read_mac_secret.data,\n\t\t  session->connection_state.read_mac_secret.size, ver);\n\

In [ ]:
import re
from openai import OpenAI

# Initialize the OpenAI client with your API key
client = OpenAI(
    api_key="sk-proj-PR4DTt7N1tlbvJ8QzTxiT3BlbkFJf2t1Sc37vC0UkbCOUhJx",
)

# Define the prompt base
prompt_base = "Analyze the following function for security vulnerabilities, if it is vulnerable respond with why if not respond with the code is clean: "

# Function to get a prediction from ChatGPT
def chat_gpt(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    text = response.choices[0].message.content.strip()
    print("Full Response:", text)
    # Define a more nuanced regular expression to determine if the function is not vulnerable
    not_vulnerable_patterns = [
        r'\bis (not|secure|safe)\b',
        r'\b(no significant security vulnerabilities)\b',
        r'\b(no|without) obvious vulnerabilities\b',
        r'\btherefore, it is not vulnerable\b',
        r'\bclean'
    ]
    # Check if any of the 'not vulnerable' patterns are matched
    if any(re.search(pattern, text, re.IGNORECASE) for pattern in not_vulnerable_patterns):
        return 0
    # If no patterns match, check for vulnerability patterns
    vulnerable_patterns = [
        r'\bvulnerable\b',
        r'\brisk\b',
        r'\bunsafe\b',
        r'\bdanger\b',
        r'\binsecure\b'
    ]
    if any(re.search(pattern, text, re.IGNORECASE) for pattern in vulnerable_patterns):
        return 1
    return 0  # Default to 0 if no clear conclusion can be drawn

In [ ]:
import random

random_functions = random.sample(corrected_data, 10)

targets = []
predictions = []

for function in random_functions:
    function_text = function["func"]
    target = function.get("target")
    if target is None:
        print("Error: No target found in JSON object")
        continue  # Skip this iteration if target is missing

    target = int(target)  # Ensure target is an integer
    prediction = chat_gpt(prompt_base + function_text)

    targets.append(target)
    predictions.append(prediction)

    # Print the entire JSON object for the function
    print("JSON Object:", json.dumps(function, indent=4))  # Pretty print JSON object
    print("Predicted:", prediction, "Actual:", target)

# Calculate and print evaluation metrics
accuracy = accuracy_score(targets, predictions)
print("Accuracy:", accuracy)

Full Response: The given function does not appear to have any obvious security vulnerabilities. The function is creating a directory using parameters passed in and then closing the file once the directory is successfully created. It does not directly deal with user input or external data that could lead to security vulnerabilities. The function seems clean in terms of security.
JSON Object: {
    "func": "NTSTATUS create_directory(connection_struct *conn, struct smb_request *req,\n\t\t\t  struct smb_filename *smb_dname)\n{\n\tNTSTATUS status;\n\tfiles_struct *fsp;\n\n\tstatus = SMB_VFS_CREATE_FILE(\n\t\tconn,\t\t\t\t\t/* conn */\n\t\treq,\t\t\t\t\t/* req */\n\t\t0,\t\t\t\t\t/* root_dir_fid */\n\t\tsmb_dname,\t\t\t\t/* fname */\n\t\tFILE_READ_ATTRIBUTES,\t\t\t/* access_mask */\n\t\tFILE_SHARE_NONE,\t\t\t/* share_access */\n\t\tFILE_CREATE,\t\t\t\t/* create_disposition*/\n\t\tFILE_DIRECTORY_FILE,\t\t\t/* create_options */\n\t\tFILE_ATTRIBUTE_DIRECTORY,\t\t/* file_attributes */\n\t\t0,\t\